# 0. Import Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv('../Data/train_short.csv')

In [3]:
df["ts_listen"] = pd.to_datetime(df["ts_listen"], unit="s")
col = ["genre_id",
       "media_id",
       "album_id",
       "context_type",
       "platform_name",
       "platform_family",
       "listen_type",
       "user_gender",
       "user_id",
       "artist_id",
       "is_listened"]

df[col] = df[col].astype("category")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   genre_id         5000 non-null   category      
 1   ts_listen        5000 non-null   datetime64[ns]
 2   media_id         5000 non-null   category      
 3   album_id         5000 non-null   category      
 4   context_type     5000 non-null   category      
 5   release_date     5000 non-null   int64         
 6   platform_name    5000 non-null   category      
 7   platform_family  5000 non-null   category      
 8   media_duration   5000 non-null   int64         
 9   listen_type      5000 non-null   category      
 10  user_gender      5000 non-null   category      
 11  user_id          5000 non-null   category      
 12  artist_id        5000 non-null   category      
 13  user_age         5000 non-null   int64         
 14  is_listened      5000 non-null   categor

In [5]:
X = pd.get_dummies(df.drop(['is_listened', 'user_id','ts_listen','release_date','media_duration','artist_id'], axis=1))
y = df['is_listened']
X.head()

,user_age,genre_id_0,genre_id_1,genre_id_3,genre_id_7,genre_id_9,genre_id_10,genre_id_14,genre_id_16,genre_id_18,...,platform_name_0,platform_name_1,platform_name_2,platform_family_0,platform_family_1,platform_family_2,listen_type_0,listen_type_1,user_gender_0,user_gender_1
0,29,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,1,0,1,0
1,30,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,0,1,0
2,29,0,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,1,0,1
3,30,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
4,24,0,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [8]:
y_train.head()

704     1
1933    1
292     1
393     0
196     1
Name: is_listened, dtype: category
Categories (2, int64): [0, 1]

# 1. Import Dependencies

In [9]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

# 2. Build and Compile Model

In [10]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [11]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

# 3. Fit, Predict and Evaluate

In [12]:
model.fit(X_train, y_train, epochs=50, batch_size=128)

Epoch 1/50
32/32 [==============================] - 1s 2ms/step - loss: 0.6541 - accuracy: 0.6327
Epoch 2/50
32/32 [==============================] - 0s 3ms/step - loss: 0.6502 - accuracy: 0.6375
Epoch 3/50
32/32 [==============================] - 0s 2ms/step - loss: 0.6490 - accuracy: 0.6375
Epoch 4/50
32/32 [==============================] - 0s 2ms/step - loss: 0.6481 - accuracy: 0.6375
Epoch 5/50
32/32 [==============================] - 0s 2ms/step - loss: 0.6476 - accuracy: 0.6375
Epoch 6/50
32/32 [==============================] - 0s 2ms/step - loss: 0.6472 - accuracy: 0.6375
Epoch 7/50
32/32 [==============================] - 0s 2ms/step - loss: 0.6469 - accuracy: 0.6375
Epoch 8/50
32/32 [==============================] - 0s 3ms/step - loss: 0.6462 - accuracy: 0.6375
Epoch 9/50
32/32 [==============================] - 0s 3ms/step - loss: 0.6457 - accuracy: 0.6375
Epoch 10/50
32/32 [==============================] - 0s 2ms/step - loss: 0.6458 - accuracy: 0.6375
Epoch 11/50
32/32 [

In [13]:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

In [14]:
accuracy_score(y_test, y_hat)

0.667

In [15]:
roc_auc_score(y_test, y_hat)

0.5173709585933068

# 4. Saving and Reloading

In [16]:
model.save('tfmodel')

INFO:tensorflow:Assets written to: tfmodel\assets


In [33]:
#del model

In [34]:
#model = load_model('tfmodel')